<a href="https://colab.research.google.com/github/DGkurnia/cnkproject/blob/main/BMLP_Derfansyah_Guswiranata_Kurnia_klasifikasi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Perkenalan diri
## Nama : Derfansyah Guswiranata Kurnia (wira)
## id dicoding: dgkurnia
## email dicoding : weerakurnia@gmail.com

# **1. Import Library**

Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning.

In [1]:
#inisiasi perpustakaan awal
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#impor data dr google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [23]:
#Perpustakaan untuk sklearn
from sklearn import metrics
from sklearn import preprocessing
from sklearn import model_selection
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler, RobustScaler

In [39]:
#impor akurasi
from sklearn.metrics import accuracy_score

In [34]:
#impor model
from sklearn.feature_selection import SelectFromModel

In [4]:
#klasifikais hutan
from sklearn.ensemble import RandomForestClassifier

# **2. Memuat Dataset dari Hasil Clustering**

Memuat dataset hasil clustering dari file CSV ke dalam variabel DataFrame.

In [6]:
#Inisiasi pengunduhan dataset dr google
clabnb = pd.read_csv('/content/drive/MyDrive/csv-files/clusairbnb.csv')
#persiapan salinan
clabnbii = clabnb.copy()
clabnbii.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48895 entries, 0 to 48894
Data columns (total 17 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              48895 non-null  int64  
 1   name                            48879 non-null  object 
 2   host_id                         48895 non-null  int64  
 3   host_name                       48874 non-null  object 
 4   neighbourhood_group             48895 non-null  object 
 5   neighbourhood                   48895 non-null  object 
 6   latitude                        48895 non-null  float64
 7   longitude                       48895 non-null  float64
 8   room_type                       48895 non-null  object 
 9   price                           48895 non-null  int64  
 10  minimum_nights                  48895 non-null  int64  
 11  number_of_reviews               48895 non-null  int64  
 12  last_review                     

In [7]:
#Persiapan segmentasi (pengabaian identitas untuk kemudahan)
clabnbii[['id','host_id']] = clabnbii[['id','host_id']].astype(str).copy()
clabnbii.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48895 entries, 0 to 48894
Data columns (total 17 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              48895 non-null  object 
 1   name                            48879 non-null  object 
 2   host_id                         48895 non-null  object 
 3   host_name                       48874 non-null  object 
 4   neighbourhood_group             48895 non-null  object 
 5   neighbourhood                   48895 non-null  object 
 6   latitude                        48895 non-null  float64
 7   longitude                       48895 non-null  float64
 8   room_type                       48895 non-null  object 
 9   price                           48895 non-null  int64  
 10  minimum_nights                  48895 non-null  int64  
 11  number_of_reviews               48895 non-null  int64  
 12  last_review                     

In [18]:
#persiapan pengujian kluster
clsinvst = clabnbii.iloc[:, 9:].copy() #cek dari harga
clsinvst = clsinvst.drop('last_review', axis=1).copy() #kolom tanggal tak usah
clsinvst.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48895 entries, 0 to 48894
Data columns (total 7 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   price                           48895 non-null  int64 
 1   minimum_nights                  48895 non-null  int64 
 2   number_of_reviews               48895 non-null  int64 
 3   reviews_per_month               48895 non-null  int64 
 4   calculated_host_listings_count  48895 non-null  int64 
 5   availability_365                48895 non-null  int64 
 6   Cluster                         48895 non-null  object
dtypes: int64(6), object(1)
memory usage: 2.6+ MB


In [26]:
#Identifikasi kluster dahulu melalui deklarasi kluster
kelcl = clsinvst['Cluster'].unique().copy()
kelcl
#ke pemecahan

array(['unit asing', 'unit kurang populer', 'opsi pelancong irit',
       'sesi unit favorit di musim tertentu',
       'bisa untuk portofolio pemilik', 'unit sangat direkomendasikan',
       'ada perencanaan dulu'], dtype=object)

In [22]:
#pemecahan
[kelcli, kelclii, kelcliii, kelcliv, kelclv, kelclvi, kelclvii] = kelcl #pemecahan menjadi satu komponen
print(kelcli) #cek satu komponen

unit asing


# **3. Data Splitting**

Tahap Data Splitting bertujuan untuk memisahkan dataset menjadi dua bagian: data latih (training set) dan data uji (test set).

In [27]:
#Inisiasi pemisahan
ycl = clsinvst['Cluster'].copy()
ycl.info()

<class 'pandas.core.series.Series'>
RangeIndex: 48895 entries, 0 to 48894
Series name: Cluster
Non-Null Count  Dtype 
--------------  ----- 
48895 non-null  object
dtypes: object(1)
memory usage: 382.1+ KB


In [29]:
#sumbu x (X)
Xcl = Xcl = clsinvst.drop('Cluster', axis=1).copy() #di luar kluster
Xcl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48895 entries, 0 to 48894
Data columns (total 6 columns):
 #   Column                          Non-Null Count  Dtype
---  ------                          --------------  -----
 0   price                           48895 non-null  int64
 1   minimum_nights                  48895 non-null  int64
 2   number_of_reviews               48895 non-null  int64
 3   reviews_per_month               48895 non-null  int64
 4   calculated_host_listings_count  48895 non-null  int64
 5   availability_365                48895 non-null  int64
dtypes: int64(6)
memory usage: 2.2 MB


In [31]:
#Numerisasi kluster dengan 'tiruan'
yclnum = pd.get_dummies(ycl).reset_index(drop=True).copy()
yclnum.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48895 entries, 0 to 48894
Data columns (total 7 columns):
 #   Column                               Non-Null Count  Dtype
---  ------                               --------------  -----
 0   ada perencanaan dulu                 48895 non-null  bool 
 1   bisa untuk portofolio pemilik        48895 non-null  bool 
 2   opsi pelancong irit                  48895 non-null  bool 
 3   sesi unit favorit di musim tertentu  48895 non-null  bool 
 4   unit asing                           48895 non-null  bool 
 5   unit kurang populer                  48895 non-null  bool 
 6   unit sangat direkomendasikan         48895 non-null  bool 
dtypes: bool(7)
memory usage: 334.4 KB


In [32]:
#Pemisahan pelatihan dan tes
Xcltrain, Xcltest, ycltrain, ycltest = train_test_split(Xcl, yclnum, test_size=0.25, random_state=40)

In [33]:
#Cek komponen
print(Xcltrain.shape)
print(Xcltest.shape)
print(ycltrain.shape)
print(ycltest.shape)

(36671, 6)
(12224, 6)
(36671, 7)
(12224, 7)


Aspek yang diperiksa
* i. Xcltrain = klasifikasi sumbu 'X' untuk latihan
* ii. Xcltest = klasifikasi sumbu 'X' untuk pengujian   
* iii. ycltrain = klasifikasi sumbu 'y' untuk latihan
* iv. ycltest = klasifikasi sumbu 'y' untuk pengujian

# **4. Membangun Model Klasifikasi**


## **a. Membangun Model Klasifikasi**

Setelah memilih algoritma klasifikasi yang sesuai, langkah selanjutnya adalah melatih model menggunakan data latih.

Berikut adalah rekomendasi tahapannya.
1. Pilih algoritma klasifikasi yang sesuai, seperti Logistic Regression, Decision Tree, Random Forest, atau K-Nearest Neighbors (KNN).
2. Latih model menggunakan data latih.

In [35]:
#Inisiasi rcf (seleksi fitur demi efisiensi ram)
pilihan = SelectFromModel(RandomForestClassifier(n_estimators=10, random_state=40))
pilihan.fit(Xcltrain, ycltrain) #pelatihan untuk Xcl dan ycl (klasifikasi X dan y)
#seleksi pelatihan
Xtrpil = pilihan.transform(Xcltrain) #sumbu X latihan
Xtspil = pilihan.transform(Xcltest) # Sumbu X bagian uji

In [36]:
#cek bentuk pilihan
print(Xtrpil.shape)
print(Xtspil.shape)

(36671, 3)
(12224, 3)


In [46]:
#Inspeksi nilai ccp alpa yang sesuai (inspeksi kisaran nilai)
nialpf = np.arange(0.001, 0.1, 5e-4) #pemeriksaan nilai
habaik = [] #cek hasil
itmaks = 100 #iterasi tertinggi
tolerance = 0.01 #toleransi
#iterasi untuk pemeriksaan

In [47]:
#iterasi nilai "ccp_alpa"
for iteration in range(itmaks):
    for alpha in nialpf:
        rclasf = RandomForestClassifier(ccp_alpha=alpha, random_state=40)
        rclasf.fit(Xcltrain, ycltrain)

        # Prediksi
        y_pred = rclasf.predict(Xcltest)

        # Evaluasi
        accuracy = accuracy_score(ycltest, y_pred)
        f1 = f1_score(ycltest, y_pred, average='samples')

        # Penyimpanan hasil
        habaik.append((alpha, accuracy, f1))
        print(f'skor ccp_alpha: {alpha:.3f}, Akurasi: {accuracy:.2f}, Nilai F1 : {f1:.2f}')

    # Langkah 3: Inspeksi hasil
    best_result = max(habaik, key=lambda x: (x[1], x[2]))
    best_alpha = best_result[0]

    print(f'Nilai ccp terbaik setelah iterasi {iteration + 1}: {best_alpha}, Akurasi: {best_result[1]:.2f}, Nilai F1 : {best_result[2]:.2f}')

    # Langkah 4: Penyesuaian sementara
    current_best_accuracy = best_result[1]
    current_best_f1 = best_result[2]

    # If the improvement is below tolerance, stop searching
    if iteration > 0 and abs(current_best_accuracy - previous_best_accuracy) < tolerance:
        print("No significant improvement. Stopping the search.")
        break

    previous_best_accuracy = current_best_accuracy

    # Penyetelan grafik
    nialpf = np.linspace(max(0.001, best_alpha - 0.005), min(0.1, best_alpha + 0.005), num=10)

# Deklarasi 'Final best result'
final_best_result = max(habaik, key=lambda x: (x[1], x[2]))
print(f'Nilai terbaik untuk ccp_alpha: {final_best_result[0]}, Akurasi: {final_best_result[1]:.2f}, Nilai F1 : {final_best_result[2]:.2f}')

skor ccp_alpha: 0.001, Akurasi: 0.97, Nilai F1 : 0.97
skor ccp_alpha: 0.002, Akurasi: 0.96, Nilai F1 : 0.96
skor ccp_alpha: 0.002, Akurasi: 0.95, Nilai F1 : 0.95
skor ccp_alpha: 0.003, Akurasi: 0.95, Nilai F1 : 0.95
skor ccp_alpha: 0.003, Akurasi: 0.95, Nilai F1 : 0.95
skor ccp_alpha: 0.004, Akurasi: 0.94, Nilai F1 : 0.94
skor ccp_alpha: 0.004, Akurasi: 0.93, Nilai F1 : 0.93
skor ccp_alpha: 0.005, Akurasi: 0.93, Nilai F1 : 0.93
skor ccp_alpha: 0.005, Akurasi: 0.92, Nilai F1 : 0.92
skor ccp_alpha: 0.006, Akurasi: 0.91, Nilai F1 : 0.91
skor ccp_alpha: 0.006, Akurasi: 0.91, Nilai F1 : 0.91
skor ccp_alpha: 0.006, Akurasi: 0.91, Nilai F1 : 0.91
skor ccp_alpha: 0.007, Akurasi: 0.91, Nilai F1 : 0.91
skor ccp_alpha: 0.008, Akurasi: 0.91, Nilai F1 : 0.91
skor ccp_alpha: 0.008, Akurasi: 0.91, Nilai F1 : 0.91
skor ccp_alpha: 0.009, Akurasi: 0.91, Nilai F1 : 0.91
skor ccp_alpha: 0.009, Akurasi: 0.91, Nilai F1 : 0.91
skor ccp_alpha: 0.010, Akurasi: 0.90, Nilai F1 : 0.90
skor ccp_alpha: 0.010, Akura

In [48]:
#Dinamisasi variabel 'Rclasifier'
rclasf = RandomForestClassifier(
    random_state=40,
    n_estimators=50,  # Efisiensi pohon
    max_depth=8,      # Pengurangan pohon demi akurasi
    min_samples_split=5,  # Pemisahan untuk bagian pohon
    min_samples_leaf=3 ,# Sampel untuk daun
    warm_start=True,  # Penyesuaian sementara
    ccp_alpha = final_best_result #nilai dari iterasi
)

## Tulis narasi atau penjelasan algoritma yang Anda gunakan.
### Urutan p

## **b. Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.
1. Lakukan prediksi menggunakan data uji.
2. Hitung metrik evaluasi seperti Accuracy dan F1-Score (Opsional: Precision dan Recall).
3. Buat confusion matrix untuk melihat detail prediksi benar dan salah.

In [ ]:
#Peningkatan periodik
for n_estimators in range(10, 101, 10):  # Awal 10 pohon akhir 100 pohon
    rclasf.n_estimators = n_estimators
    rclasf.fit(Xcltrain, ycltrain)

    # Prediksis
    y_pred = rclasf.predict(Xcltest)

    # Evaluate model performance
    accuracy = accuracy_score(ycltest, y_pred)
    f1 = f1_score(ycltest, y_pred)

    print(f'Total Pohon: {n_estimators}, Akurasi: {accuracy:.2f}, Nilai F1 : {f1:.2f}')

    # Check if performance meets requirements
    if accuracy >= 0.87 and f1 >= 0.87:
        print("Model sudah memenuhi nilai metrik.")
        break
    elif accuracy >= 0.87 and f1 < 0.87:
        print("Akurasi sudah memenuhi, tetapi Nilai F1 belum memenuhi.")
    elif accuracy < 0.87 and f1 >= 0.87:
        print("Nilai F1 sudah memenuhi, tetapi akurasi belum memenuhi.")
else:
    print("Model tidak memenuhi metrik yang diperlukan dengan pengaturan yang diberikan.")

Tulis hasil evaluasi algoritma yang digunakan, jika Anda menggunakan 2 algoritma, maka bandingkan hasilnya.

## **c. Tuning Model Klasifikasi (Optional)**

Gunakan GridSearchCV, RandomizedSearchCV, atau metode lainnya untuk mencari kombinasi hyperparameter terbaik

In [ ]:
#Inisiasi gridsearch

In [ ]:
#Type your code here

## **d. Evaluasi Model Klasifikasi setelah Tuning (Optional)**

Berikut adalah rekomendasi tahapannya.
1. Gunakan model dengan hyperparameter terbaik.
2. Hitung ulang metrik evaluasi untuk melihat apakah ada peningkatan performa.

In [ ]:
#Type your code here

## **e. Analisis Hasil Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.
1. Bandingkan hasil evaluasi sebelum dan setelah tuning (jika dilakukan).
2. Identifikasi kelemahan model, seperti:
  - Precision atau Recall rendah untuk kelas tertentu.
  - Apakah model mengalami overfitting atau underfitting?
3. Berikan rekomendasi tindakan lanjutan, seperti mengumpulkan data tambahan atau mencoba algoritma lain jika hasil belum memuaskan.